Use wind as a control parameter
==================

Jinbo Wang

Jinbo.Wang@jpl.nasa.gov

last update: 12/31/2021

copied from full_wacm-scenario-full-year.ipynb

In [ ]:
import slab_model_varying_winds as slab_model
import pylab as plt
import numpy as np
import xarray as xr
from popy import utils
from scipy.io import loadmat
import pandas as pd
from scipy.interpolate import interp1d

xrod=xr.open_dataset

# WaCM was generated from the original

Highpass filtering and detides will be performed on the simulated WaCM data.

In [ ]:
import slab_model_varying_winds as slab_model
import pylab as plt
import numpy as np
import xarray as xr
from popy import utils
from scipy.io import loadmat
import pandas as pd


xrod=xr.open_dataset
t0,t1='2007-09-20','2007-09-29'

#load data station papa
fn='../data/from_Hong/PaPa2.mat'
lat0=50.01
tt=pd.date_range('2007-06-08 04:00:00',periods=8541,freq='1h')


#fn='../data/from_Hong/KEO2.mat'
#lat0=32.3
#tt=pd.date_range('2007-09-26 11:00:00',periods=8255,freq='1h')
#t0,t1='2007-11-20','2007-12-20'
#t0,t1='2007-12-13','2008-01-30'

d=loadmat(fn)

#load velocity and create xarrays 
vobs=xr.DataArray(d['VV_obs'][:].flatten(),dims=('time'),coords={'time':tt} )
uobs=xr.DataArray(d['UU_obs'][:].flatten(),dims=('time'),coords={'time':tt} )
#load winds and create xarrays
vwobs=xr.DataArray(d['Vwind_o'][:].flatten(),dims=('time'),coords={'time':tt} )#.sel(time=slice(t0,t1))
uwobs=xr.DataArray(d['Uwind_o'][:].flatten(),dims=('time'),coords={'time':tt} )#.sel(time=slice(t0,t1))

orbit='500_1000'
orbit='700_1800'
#orbit='hourly'
#orbit='11h'

d700=slab_model.synthetic_wacm(uobs,vobs,uwobs,vwobs,lat0,
                               cutoff_truth=1/3,cutoff_wacm=1/20,
                               orbit=orbit)


## Focus on one day, use ensembles with different initial time

In [ ]:
t_center='2007-09-15'
fig,ax=plt.subplots(3,1,figsize=(20,10),sharex=True)
time=d700.u_wacm_total.time.values[10:]

for i in range(1):
    for j in np.arange(0,time.size-1,4)[:1]:
        t0=time[j];t1=time[j+20]
        t0=np.datetime64('2007-09-22')
        t1=np.datetime64('2007-09-29')
        
        ut=d700.u_wacm_total.sel(time=slice(t0,t1))
        
        t0=ut.time.data[0]
        t1=ut.time.data[-1]
        
        ut.plot(ax=ax[2],color='purple',marker='o')
        
        up,vp,param=d700.get_wacm_nio(t0,t1,use_hourly_wind=True)
        up.plot(ax=ax[0],label='%i'%i,marker='+')
        ut0= (ut -d700.u_truth_high_tide.interp(time=ut.time.values) 
              - up.interp(time=ut.time.values)).interp(time=up.time.values)+up+d700.u_truth_high_tide.interp(time=up.time.values)
        ut0.plot(ax=ax[2],color='r',)
        
        up,vp,param=d700.get_wacm_nio(t0,t1,use_hourly_wind=False)
        up.plot(ax=ax[1],label='%i'%i,marker='+')
        ut0= (ut -d700.u_truth_high_tide.interp(time=ut.time.values) 
              - up.interp(time=ut.time.values)).interp(time=up.time.values)+up+d700.u_truth_high_tide.interp(time=up.time.values)
        ut0.plot(ax=ax[2],color='g',)
        
        d700.u_truth_high_notide_nio.sel(time=slice(t0,t1)).plot(ax=ax[0],color='k',lw=3)
        d700.u_wacm_high.sel(time=slice(t0,t1)).plot(ax=ax[0],color='k',marker='o')
        
        d700.u_truth_high_notide_nio.sel(time=slice(t0,t1)).plot(ax=ax[1],color='k',lw=3)
        d700.u_wacm_high.sel(time=slice(t0,t1)).plot(ax=ax[1],color='k',marker='o')
        d700.u_truth_high_notide.sel(time=slice(t0,t1)).plot(ax=ax[1],color='k',marker='+')
        
        d700.u_truth_total.sel(time=slice(t0,t1)).plot(ax=ax[2],color='k')
        
ax[0].legend()

In [ ]:
#loop through every ten days
u,v,H,c=[],[],[],[]

ttend=d700.u_wacm_total[-1].time.data

total_days=uobs.size//24

for t00 in d700.u_wacm_total[:4].time.values:
    u_wacm_nio=uobs*0
    v_wacm_nio=vobs*0
    H_wacm=uobs*0
    c_wacm=uobs*0
    #print(i,np.datetime64('now'))
    #for ii in range(i,8541-240,240)[:5]: 
    print(t00)
    print(pd.date_range(t00,'2008-05-28T21:20:00.000000000',freq='6D'))
    for it0 in np.range(0,total_days,6*24):
        t0=t00+np.timedelta64(it0,'h')
        t1=t0+np.timedelta64(it0+6*24,'h')
        t_out=u_wacm_nio.sel(time=slice(t0,t1)).data[:]
        up,vp,param=d700.get_wacm_nio(t0,t1,use_hourly_wind=True)
        u_wacm_nio.sel(time=slice(t0,t1)).data[:]=up.interp(time=t_out).data[:]
        v_wacm_nio.sel(time=slice(t0,t1)).data[:]=vp.interp(time=t_out).data[:]
        #H_wacm[ii:ii+240].data[:]=param['H']
        #c_wacm[ii:ii+240].data[:]=param['c']
    u.append(u_wacm_nio)
    v.append(v_wacm_nio)
    #H.append(H_wacm)
    #c.append(c_wacm)
    

In [ ]:
fig,ax=plt.subplots(3,1,figsize=(10,10))
d700.u_truth_high_notide_nio.sel(time=slice('2007-09-20 00:00:00',t1)).plot(ax=ax[0],color='k',lw=3)

for uu in u:
    uu.plot(ax=ax[0])


In [ ]:
uu=xr.DataArray(np.zeros((24,u[0].size)), dims=('case','time'),
                coords={'case':range(24),'time':u[0].time.values})
vv=xr.DataArray(np.zeros((24,u[0].size)), dims=('case','time'),
                coords={'case':range(24),'time':u[0].time.values})

for i in range(4):
    uu[i,:]=u[i].interp(time=u[0].time.values)
    vv[i,:]=v[i].interp(time=u[0].time.values)

fig,ax=plt.subplots(3,1,figsize=(10,10))

tt0,tt1='2007-06-20','2007-06-29'
for i in range(4):
    #uu[i,:].sel(time=slice(tt0,tt1)).plot(ax=ax[0],color='gray')
    u[i].sel(time=slice(tt0,tt1)).plot(ax=ax[0])
    d700.u_truth_high_notide_nio.sel(time=slice(tt0,tt1)).plot(ax=ax[0],color='gray',lw=3)
    
uu=d700.u_truth_high_notide_nio
vv=d700.v_truth_high_notide_nio


((uu**2+vv**2)**0.5).resample(time='1D').mean().plot(color='red',lw=1)

In [ ]:
((u_wacm_nio**2+v_wacm_nio**2)**0.5).resample(time='1D').mean().plot(label='wacm')
((d700.u_truth_high_notide_nio**2+d700.v_truth_high_notide_nio**2)**0.5).resample(time='1D').mean().plot(label='true')
plt.legend()